In [1]:
import datetime
import numpy as np
from matplotlib import pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import pandas as pd
import talib as ta

In [2]:
win = pd.read_excel('INDICES.xlsx', sheet_name='WIN')

In [3]:
win = win.dropna()

In [4]:
bop = ta.BOP(win.Open, win.Max, win.Min, win.Close)

In [5]:
bop

0        0.303030
1        0.666667
2       -0.533333
3       -0.400000
4        0.750000
           ...   
10829    0.000000
10830    0.000000
10831    0.000000
10832    0.000000
10833    0.000000
Length: 10834, dtype: float64

In [65]:
tAlta = []
tBaixa = []
maxTend = 0
minTend = 0
tendencia = '0'
profit = pd.DataFrame(columns=['DataEnt','Entrada','DataSai','Saida', 'Operacao','Saldo'])
posicionado = 0 # 1 - Sim, 0 - Não
volMinOper = 500000000
vlrMinDifMedias = 100


In [7]:
win.columns

Index(['Data', 'Open', 'Max', 'Min', 'Close', 'Media9', 'Media21', 'Media200',
       'Vol'],
      dtype='object')

In [58]:
profit.columns = [None]

ValueError: Length mismatch: Expected axis has 5 elements, new values have 1 elements

In [66]:
profit

,DataEnt,Entrada,DataSai,Saida,Operacao,Saldo


In [68]:
def getMinValue(vlr1 : float, vlr2 : float):
    if(vlr1 <= vlr2):
        return vlr1
    else:
        return vlr2
    
def getMaxValue(vlr1 : float, vlr2 : float):
    if(vlr1 >= vlr2):
        return vlr1
    else:
        return vlr2    

In [ ]:
ty(tAlta)

In [72]:
for i in win.index:
    tBaixa = win.Close[i]
    tAlta = win.Close[i]
    if(win.Media21[i] > win.Media9[i]): #identificando a tendencia a partir da diferença das medias
        if(tendencia == 'Baixa'): # identificar alternancia de tendencias
            tBaixa.append(win.Close[i])
            if(posicionado == 0):
                if(win.Close[i] < getMinValue(minTend, maxTend) and (win.Media21[i] - win.Media9[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper): #localizando oportunidade de compra a partir da comparacao do vlr atual e o fundo conhecido da tendencia
                    profit.loc[profit.index.size, ['DataEnt']] = win.Data[i]
                    profit.loc[profit.index.size, ['Entrada']] = win.Close[i]
                    profit.loc[profit.index.size, ['Operacao']] = 'Venda'
                    posicionado = 1
                    minTend = min(tBaixa)
                else:
                    minTend = min(tBaixa)
            elif(posicionado == 1):
                if(win.Close[i] < getMinValue(minTend, maxTend) and (win.Media21[i] - win.Media9[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper):
                    minTend = min(tBaixa)
                else:
                    profit.loc[profit.index.size -1, ['DataSai']] = win.Data[i]
                    profit.loc[profit.index.size -1, ['Saida']] = win.Close[i]
                    posicionado = 0
                    minTend = min(tBaixa)
        else:
            tendencia = 'Baixa'
            maxTend = min(tAlta)
            minTend = win.Close[i]
            tBaixa = []
            i = i-1
    elif(win.Media21[i] < win.Media9[i]):
        if(tendencia == 'Alta'): # identificar alternancia de tendencias
            tAlta.append(win.Close[i])
            if(posicionado == 0):
                if(win.Close[i] > getMiaxValue(minTend, maxTend) and (win.Media9[i] - win.Media21[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper): #localizando oportunidade de compra a partir da comparacao do vlr atual e o fundo conhecido da tendencia
                    profit.loc[profit.index.size, ['DataEnt']] = win.Data[i]
                    profit.loc[profit.index.size, ['Entrada']] = win.Close[i]
                    profit.loc[profit.index.size, ['Operacao']] = 'Compra'
                    posicionado = 1
                    maxTend = max(tAlta)
                else:
                    maxTend = max(tAlta)
            elif(posicionado == 1):
                if(win.Close[i] > getMaxValue(minTend, maxTend) and (win.Media9[i] - win.Media21[i]) > vlrMinDifMedias and win.Vol[i] > volMinOper):
                    maxTend = max(tAlta)
                else:
                    profit.loc[profit.index.size -1, ['DataSai']] = win.Data[i]
                    profit.loc[profit.index.size -1, ['Saida']] = win.Close[i]
                    posicionado = 0
                    maxTend = max(tAlta)
        else:
            tendencia = 'Alta'
            maxTend = win.Close[i]
            minTend = max(tBaixa)
            tAlta = []
            i = i-1
        

AttributeError: 'numpy.int64' object has no attribute 'append'

In [10]:
tAlta

[123460,
 123390,
 123430,
 123445,
 123335,
 123315,
 123300,
 123320,
 123325,
 123290,
 123295,
 123320,
 123210,
 123210,
 123265,
 123290,
 123210,
 123110,
 123160,
 123130,
 123085,
 123135,
 123020,
 123045,
 123090,
 123135,
 123015,
 123220,
 123320,
 123310,
 123245,
 123305,
 123225,
 123280,
 123300,
 123265,
 123090,
 123055,
 123025,
 123140,
 123050,
 122930,
 122930,
 122795,
 122700,
 122870,
 123030,
 123040,
 122920,
 122840,
 122830,
 122820,
 122975,
 123065,
 123650,
 123755,
 123805,
 123835,
 123855,
 123815,
 123760,
 123770,
 123755,
 123720,
 123685,
 123710,
 123675,
 123665,
 123675,
 123650,
 123700,
 123755,
 123785,
 123775,
 123835,
 123835,
 123815,
 123825,
 123820,
 123645,
 123965,
 123955,
 123935,
 123995,
 123960,
 123975,
 123945,
 123940,
 124010,
 123995,
 123980,
 124000,
 124000,
 124000,
 123980,
 123905,
 123980,
 123915,
 124010,
 124035,
 123955,
 124045,
 124125,
 124080,
 124030,
 124100,
 124095,
 124050,
 124045,
 124070,
 124090,
 